In [135]:
import math 

Nodes = 10
Cores_per_Node = 16
Ram_per_Node = 64 # in GB
num_executors = 5 # valude needs to be x <= 5


# Nodes = 5
# Cores_per_Node = 8
# Ram_per_Node = 32 # in GB
# num_executors = 3 # valude needs to be x <= 5

# Nodes = 4
# Cores_per_Node = 12
# Ram_per_Node = 48 # in GB
# num_executors = 4 # valude needs to be x <= 5


print(f"No of Core Per Node =>  {Cores_per_Node}")
print(f"Leave One Core per Node for Hadoop/Yarn decmon {Cores_per_Node} - 1 (and RAM 1 GB) => {Cores_per_Node-1} ")
Cores_per_Node-=1

print(f"So Avaliable RAM is {Ram_per_Node} GB -1 => {Ram_per_Node-1}")
Ram_per_Node-=1

print(f"Total Avaliable Cores in Cluster [{Nodes} x {Cores_per_Node}] => {Nodes*(Cores_per_Node)}")
Total_cores = Nodes*(Cores_per_Node)
print(f"Lets Assume we go with {num_executors} (executors valude needs to be x <= 5)")
print(f"No of Available Executors [{Total_cores} / {num_executors}] => {math.floor(Total_cores/num_executors)} ")
Available_Executors = math.floor(Total_cores/num_executors)

print(f"No of Executor per Node [{Available_Executors}/{Nodes}] => {math.floor(Available_Executors/Nodes)} ")
executors_per_node = math.floor(Available_Executors/Nodes)
print(f"Leaving 1 Executor for Application Management [{Available_Executors}-1] => {math.floor(Available_Executors-1)} ")
no_of_executors = math.floor(Available_Executors-1)
#executors_per_node = math.floor(Available_Executors/Nodes)

print(f"Memory Per executor [{Ram_per_Node}/{executors_per_node}] => {math.floor(Ram_per_Node/executors_per_node)}")
Memory_per_executor = math.floor(Ram_per_Node/executors_per_node)
print(f"Counting off Heap over head 14% approx [{Memory_per_executor} * 0.14] => { math.ceil(Memory_per_executor*0.14) }   ")
head_over_heap = math.ceil(Memory_per_executor*0.14)
print(f"So actual executor memory [ {Memory_per_executor} - {head_over_heap} ]=> {Memory_per_executor -  head_over_heap } ")
final_executor_memory = Memory_per_executor -  head_over_heap


print(f'''
spark-submit \
--num-executors {no_of_executors} \
--executor-cores {num_executors} \
--executor-memory {final_executor_memory }G \
--conf spark.yarn.executor.memoryOverhead={head_over_heap}G \
--driver-memory 12G \
your_app.py
''')


No of Core Per Node =>  16
Leave One Core per Node for Hadoop/Yarn decmon 16 - 1 (and RAM 1 GB) => 15 
So Avaliable RAM is 64 GB -1 => 63
Total Avaliable Cores in Cluster [10 x 15] => 150
Lets Assume we go with 5 (executors valude needs to be x <= 5)
No of Available Executors [150 / 5] => 30 
No of Executor per Node [30/10] => 3 
Leaving 1 Executor for Application Management [30-1] => 29 
Memory Per executor [63/3] => 21
Counting off Heap over head 14% approx [21 * 0.14] => 3   
So actual executor memory [ 21 - 3 ]=> 18 

spark-submit --num-executors 29 --executor-cores 5 --executor-memory 18G --conf spark.yarn.executor.memoryOverhead=3G --driver-memory 12G your_app.py



## Is there any logic to calculate driver memory ?

there is a practical logic to estimating driver memory, although it's a bit more art than strict science. Let's break it down so you can apply it with confidence:

🧠 What the Driver Does
The Spark driver:
- Coordinates execution across the cluster
- Maintains metadata and DAGs
- Collects and logs results (especially in actions like .collect() or .count())
If your job involves:
- Heavy broadcasting
- Large metadata (many partitions, complex DAGs)
- Big result sets returned to the driver
→ Then it needs more memory.

📏 Heuristic for Driver Memory Allocation
There's no universal formula, but here are smart heuristics based on your executor memory and job profile:
| Scenario | Driver Memory Tip | 
| Small to moderate jobs | Match executor memory | 
| Medium/large jobs with caching/shuffles | Executor memory + 20–50% | 
| Jobs using .collect() on big data | 2× executor memory or more | 
| Jobs in cluster mode (YARN/K8s) | At least 2–4 GB reserved minimum | 


💡 In your recent config:
- Executors: 19 GB memory
- You’ve chosen: --driver-memory 12G
That's a conservative and generally safe choice—especially if you aren't collecting huge datasets. But if you're noticing slow performance or OOM errors on the driver, try bumping to 16–20 GB.

🔧 Optional Spark Tuning Flags
You can also set:
--conf spark.driver.memoryOverhead=2G


If the driver is used heavily, like in streaming apps or complex joins, this ensures more buffer space beyond the heap.

Want to build a small utility or config calculator that suggests driver memory based on job type and executor setup? That could be a powerful scripting aid for future deployments 💥
